In [1]:
import sys 
sys.path.append('../')
import numpy as np
import torch
import torch.utils.data
from torch import optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from models import * 
from loss_functions import *
import os 
os.chdir("/ContinuousBernoulliVAE/notebooks")

IMAGE_PATH = "../images/b2/"
MODEL_PATH = "../trained_models/"
DIM = 2
EPOCHS = 100
torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False,
                   transform=transforms.ToTensor()),
    batch_size=128, shuffle=False)

model = VAE2().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train(epoch):
    model.train()
    train_loss = 0
    train_loss_vals = []
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = vae_loss(recon_batch, data, mu, logvar)
        train_loss_vals.append(sum([temp.item() for temp in loss]))
        loss = sum(loss[:1])
        loss.backward()
        train_loss += loss.item()

        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))
    train_loss /= len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss))
    return np.array(train_loss_vals) / len(train_loader.dataset) * len(train_loader)


def test(epoch):
    model.eval()
    test_loss = 0
    test_loss_vals = []
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)

            loss = vae_loss(recon_batch, data, mu, logvar)
            test_loss_vals.append(sum([temp.item() for temp in loss]))
            loss = sum(loss[:1])
            test_loss += loss

            if i == 0:
                n = min(data.size(0), 8)
                recon_batch = recon_batch.view(128, 1, 28, 28)
                comparison = torch.cat([data[:n],
                                        recon_batch[:n]])

                save_image(comparison.cpu(),
                           f'{IMAGE_PATH}/reconstruction_' + str(epoch) + '.png', nrow=n)
                # plt.figure(figsize=(10, 4))
                # for i in range(1, 2*n+1):
                #     ax = plt.subplot(2, n, i)
                #     plt.imshow(comparison.cpu().detach().numpy()
                #                [i-1, 0, :, :], cmap="gray")
                #     ax.get_xaxis().set_visible(False)
                #     ax.get_yaxis().set_visible(False)
                #     ax.margins(0, 0)
                # plt.savefig('results/reconstruction_' + str(epoch) + '.png')
                # plt.close()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    return np.array(test_loss_vals) / len(test_loader.dataset) * len(test_loader)


train_loss_vals_total = np.array([])
test_loss_vals_total = np.array([])
for epoch in range(1, EPOCHS + 1):
    train_loss = train(epoch)
    train_loss_vals_total = np.append(train_loss_vals_total, train_loss)
    test_loss = test(epoch)
    test_loss_vals_total = np.append(test_loss_vals_total, test_loss)
    with torch.no_grad():
        sample = torch.randn(64, DIM).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   f'{IMAGE_PATH}/sample_' + str(epoch) + '.png')

torch.save(model, f'{MODEL_PATH}/vae2.pt')



Train Epoch: 1 [0/60000 (0%)]	Loss: 546.580200
Train Epoch: 1 [1280/60000 (2%)]	Loss: 228.033112
Train Epoch: 1 [2560/60000 (4%)]	Loss: 219.602692
Train Epoch: 1 [3840/60000 (6%)]	Loss: 207.913742
Train Epoch: 1 [5120/60000 (9%)]	Loss: 210.123138
Train Epoch: 1 [6400/60000 (11%)]	Loss: 206.372742
Train Epoch: 1 [7680/60000 (13%)]	Loss: 205.739685
Train Epoch: 1 [8960/60000 (15%)]	Loss: 200.115692
Train Epoch: 1 [10240/60000 (17%)]	Loss: 200.748413
Train Epoch: 1 [11520/60000 (19%)]	Loss: 188.966080
Train Epoch: 1 [12800/60000 (21%)]	Loss: 190.942764
Train Epoch: 1 [14080/60000 (23%)]	Loss: 196.020096
Train Epoch: 1 [15360/60000 (26%)]	Loss: 186.254532
Train Epoch: 1 [16640/60000 (28%)]	Loss: 183.063568
Train Epoch: 1 [17920/60000 (30%)]	Loss: 189.361633
Train Epoch: 1 [19200/60000 (32%)]	Loss: 174.777222
Train Epoch: 1 [20480/60000 (34%)]	Loss: 179.421875
Train Epoch: 1 [21760/60000 (36%)]	Loss: 176.775482
Train Epoch: 1 [23040/60000 (38%)]	Loss: 185.736588
Train Epoch: 1 [24320/60000 